In [1]:
# ===============================
# SYMBOL DETECTION NOTEBOOK
# (NO OCR, NO ML)
# ===============================

import cv2
import numpy as np
import matplotlib.pyplot as plt


# -------------------------------
# Display helper
# -------------------------------
def show(img, title=""):
    plt.figure(figsize=(3,3))
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis("off")
    plt.show()


# -------------------------------
# Preprocessing
# -------------------------------
def preprocess(cell):
    cell = cv2.cvtColor(cell, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(cell, (7,7), 0)
    _, bin_img = cv2.threshold(
        blur, 0, 255,
        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )
    return bin_img


# -------------------------------
# Empty cell detection
# -------------------------------
def is_empty(bin_img):
    ink_ratio = cv2.countNonZero(bin_img) / bin_img.size
    return ink_ratio < 0.08


# -------------------------------
# Hough line detection
# -------------------------------
def detect_lines(bin_img):
    edges = cv2.Canny(bin_img, 50, 150)
    lines = cv2.HoughLinesP(
        edges,
        rho=1,
        theta=np.pi / 180,
        threshold=40,
        minLineLength=20,
        maxLineGap=5
    )
    return lines


# -------------------------------
# Count vertical & horizontal lines
# -------------------------------
def count_lines(lines):
    v, h = 0, 0

    if lines is None:
        return v, h

    for l in lines:
        x1,y1,x2,y2 = l[0]
        angle = abs(np.degrees(np.arctan2(y2-y1, x2-x1)))

        if angle < 15:
            h += 1
        elif abs(angle - 90) < 15:
            v += 1

    return v, h


# -------------------------------
# Detect CROSS (✗)
# -------------------------------
def is_cross(lines):
    if lines is None or len(lines) < 2:
        return False

    angles = []
    for l in lines:
        x1,y1,x2,y2 = l[0]
        angle = abs(np.degrees(np.arctan2(y2-y1, x2-x1)))
        angles.append(angle)

    return (
        any(30 < a < 60 for a in angles) and
        any(120 < a < 150 for a in angles)
    )


# -------------------------------
# Detect TICK (✓)
# -------------------------------
def is_tick(lines):
    if lines is None or len(lines) < 2:
        return False

    lengths = []
    angles = []

    for l in lines:
        x1,y1,x2,y2 = l[0]
        length = np.hypot(x2-x1, y2-y1)
        angle = abs(np.degrees(np.arctan2(y2-y1, x2-x1)))
        lengths.append(length)
        angles.append(angle)

    if len(lengths) < 2:
        return False

    short = min(lengths)
    long = max(lengths)

    return (
        short / long < 0.6 and
        any(20 < a < 70 for a in angles) and
        any(100 < a < 160 for a in angles)
    )


# -------------------------------
# Detect QUESTION (?)
# -------------------------------
def is_question(bin_img):
    cnts,_ = cv2.findContours(
        bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    if len(cnts) < 2:
        return False

    areas = sorted([cv2.contourArea(c) for c in cnts])
    return areas[-1] > 5 * areas[0]


# -------------------------------
# FINAL SYMBOL DETECTOR
# -------------------------------
def detect_symbol(cell):
    bin_img = preprocess(cell)

    if is_empty(bin_img):
        return "EMPTY", ""

    lines = detect_lines(bin_img)
    v, h = count_lines(lines)

    if v > 0 and h == 0:
        return "VERTICAL_LINES", v

    if h > 0 and v == 0:
        return "HORIZONTAL_LINES", 5 - h

    if is_cross(lines):
        return "CROSS", 0

    if is_tick(lines):
        return "TICK", 5

    if is_question(bin_img):
        return "QUESTION", ""

    return "UNKNOWN", ""


# ===============================
# TEST
# ===============================
cell = cv2.imread("../data/cells/1/row2_cell5.jpg", 0)  # <-- change path
symbol, value = detect_symbol(cell)

print("Detected symbol:", symbol)
print("Output value:", value)

show(preprocess(cell), "Processed Cell")


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
